In [1]:
import pandas as pd

In [5]:
df = pd.read_parquet('/home/dmoreno/pipeline_v4_final/pipeline/training/stamp_classifier/data_acquisition/rubin/data/processed/partitions/ts_stamps_v0.0.4_dp1')
df

,diaObjectId,diaSourceId,class,partition
0,628760465987600423,600425736871346826,VS,test
1,611255003922825940,600320190600183907,AGN,test
2,650018836115882038,600386155389124609,asteroid,test
3,592915218690998602,600359786038427673,SN,test
4,648367644528869531,600386157807140919,asteroid,test
...,...,...,...,...
24235,604062754768160203,600390560818659484,VS,validation_4
24236,592915356129952775,600416938765058728,bogus,validation_4
24237,648375547268694077,600386155389124652,asteroid,validation_4
24238,614429019114315195,600430118274859452,bogus,validation_4


In [6]:
df.sample(50)

,diaObjectId,diaSourceId,class,partition
18542,579583227887354659,600394931885834300,VS,training_3
2708,648374722634973387,600408138510237722,asteroid,training_0
12260,611255072642302474,600320193014530099,AGN,training_2
3088,628769880555918070,600460943387263380,bogus,training_0
15970,648365445505614049,600403728651517957,asteroid,training_3
4411,648366888614625581,600399336110555171,asteroid,training_0
6679,614429637589598470,600368581858820276,bogus,training_1
3892,630406709772298313,600465369215271980,bogus,training_0
19125,614437746487853622,600368583069401107,bogus,validation_3
13539,604063373243449407,600390553703547286,VS,training_2


In [12]:
import pandas as pd


# Muestra el conteo de clases POR PARTICIÓN en los datos originales para comparar
print("--- Conteo de clases por partición (ANTES) ---")
print(df.groupby('partition')['class'].value_counts())
print("-" * 45)

# --- INICIO DEL PROCESO DE UNDERSAMPLING ---

# 1. Separar los datos que NO se tocarán (validation y test)
holdout_df = df[~df['partition'].str.startswith('training')].copy()

# 2. Seleccionar solo los datos de entrenamiento para submuestrear
training_data_to_sample = df[df['partition'].str.startswith('training')].copy()

# 3. Lista para guardar los DataFrames de cada partición de training ya procesada
undersampled_partitions = []

# 4. Iterar sobre cada partición de training única
for part in training_data_to_sample['partition'].unique():
    
    # Filtrar el DataFrame para obtener solo la partición de training actual
    current_partition_df = training_data_to_sample[training_data_to_sample['partition'] == part]
    
    # Calcular el número de muestras en la clase minoritaria de ESTA partición
    min_count = current_partition_df['class'].value_counts().min()
    
    print(f"Procesando '{part}': La clase minoritaria tiene {min_count} objetos. Todas las clases se reducirán a este tamaño.")
    
    # Agrupar por clase y tomar una muestra aleatoria de tamaño 'min_count' de cada una
    undersampled_partition = current_partition_df.groupby('class', group_keys=False).apply(
        lambda x: x.sample(min_count, random_state=42)
    )
    
    # Añadir el resultado a nuestra lista
    undersampled_partitions.append(undersampled_partition)

# 5. Combinar todas las particiones de training submuestreadas en un solo DataFrame
undersampled_training_df = pd.concat(undersampled_partitions)

# 6. Finalmente, unir los datos de training submuestreados con los datos de holdout (validation y test) originales
final_df = pd.concat([undersampled_training_df, holdout_df])

# --- FIN DEL PROCESO ---


# Muestra el conteo de clases en el DataFrame final para verificar
print("\n" + "-" * 45)
print("--- Conteo de clases por partición (DESPUÉS) ---")
print(final_df.groupby('partition')['class'].value_counts().sort_index())
print("-" * 45)

print("\nShape original del DataFrame:", df.shape)
print("Shape final del DataFrame:", final_df.shape)

--- Conteo de clases por partición (ANTES) ---
partition     class    
test          VS           454
              bogus        400
              asteroid     200
              AGN           52
              satellite     40
                          ... 
validation_4  bogus        320
              asteroid     160
              AGN           41
              satellite     32
              SN             7
Name: count, Length: 66, dtype: int64
---------------------------------------------
Procesando 'training_0': La clase minoritaria tiene 29 objetos. Todas las clases se reducirán a este tamaño.
Procesando 'training_1': La clase minoritaria tiene 28 objetos. Todas las clases se reducirán a este tamaño.
Procesando 'training_2': La clase minoritaria tiene 29 objetos. Todas las clases se reducirán a este tamaño.
Procesando 'training_3': La clase minoritaria tiene 29 objetos. Todas las clases se reducirán a este tamaño.
Procesando 'training_4': La clase minoritaria tiene 29 objetos. Toda

/tmp/ipykernel_2015236/4233782608.py:32: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  undersampled_partition = current_partition_df.groupby('class', group_keys=False).apply(
/tmp/ipykernel_2015236/4233782608.py:32: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  undersampled_partition = current_partition_df.groupby('class', group_keys=False).apply(
/tmp/ipykernel_2015236/4233782608.py:32: FutureWarning: DataFrameGroup

In [17]:
final_df[final_df.partition == 'training_0']['class'].value_counts()

class
AGN          29
SN           29
VS           29
asteroid     29
bogus        29
satellite    29
Name: count, dtype: int64

In [18]:
final_df[final_df.partition == 'validation_0']['class'].value_counts()

class
VS           364
bogus        320
asteroid     160
AGN           41
satellite     32
SN             7
Name: count, dtype: int64

In [19]:
final_df[final_df.partition == 'test']['class'].value_counts()

class
VS           454
bogus        400
asteroid     200
AGN           52
satellite     40
SN             9
Name: count, dtype: int64

In [20]:
final_df.to_parquet('ajam.parquet')

In [24]:
import pandas as pd
import io
import random

MAX_RATIO_OVER_MIN = 2.5 

print("--- Conteo de clases por partición (ANTES) ---")
print(df.groupby('partition')['class'].value_counts())
print("-" * 45)

# --- INICIO DEL PROCESO DE UNDERSAMPLING ---

# 1. Separar datos que no se tocarán (validation y test)
holdout_df = df[~df['partition'].str.startswith('training')].copy()
training_data = df[df['partition'].str.startswith('training')].copy()

undersampled_partitions = []

# 2. Iterar sobre cada partición de training
for part in training_data['partition'].unique():
    current_partition_df = training_data[training_data['partition'] == part]
    
    class_counts = current_partition_df['class'].value_counts()
    min_count = class_counts.min()
    minority_class_name = class_counts.idxmin()
    
    print(f"Procesando '{part}': La clase minoritaria '{minority_class_name}' tiene {min_count} objetos.")
    
    # Lista para guardar los dataframes de cada clase (ya muestreados) de esta partición
    processed_classes_for_partition = []
    
    # 3. Iterar sobre cada clase DENTRO de la partición actual
    for class_name, current_class_count in class_counts.items():
        class_df = current_partition_df[current_partition_df['class'] == class_name]
        
        # Si es la clase minoritaria, la guardamos completa
        if class_name == minority_class_name:
            target_size = current_class_count
            print(f"... Clase '{class_name}' ({current_class_count} obj): Manteniendo todos los objetos (es la minoritaria).")
            processed_classes_for_partition.append(class_df)
            continue # Pasar a la siguiente clase

        # Si es una clase mayoritaria, calculamos el nuevo tamaño
        # Límite inferior es el tamaño de la clase minoritaria
        lower_bound = min_count
        # Límite superior es un % por encima de la minoritaria, pero NUNCA más grande que la cantidad de datos que ya tenemos
        upper_bound = min(int(min_count * MAX_RATIO_OVER_MIN), current_class_count)
        
        # Elige un tamaño aleatorio dentro del rango [lower, upper]
        # Aseguramos que el lower_bound no sea mayor que el upper_bound
        if lower_bound >= upper_bound:
             target_size = upper_bound
        else:
             target_size = random.randint(lower_bound, upper_bound)
        
        print(f"... Clase '{class_name}' ({current_class_count} obj): Reduciendo a un tamaño aleatorio en [{lower_bound}, {upper_bound}] -> {target_size} objetos.")

        # Realizamos el muestreo y lo guardamos
        sampled_class_df = class_df.sample(n=target_size, random_state=42)
        processed_classes_for_partition.append(sampled_class_df)
        
    # Combinamos todas las clases procesadas para esta partición
    undersampled_partition = pd.concat(processed_classes_for_partition)
    undersampled_partitions.append(undersampled_partition)

# 4. Combinar todas las particiones de training ya procesadas
undersampled_training_df = pd.concat(undersampled_partitions)

# 5. Unir con los datos de holdout (validation y test)
final_df = pd.concat([undersampled_training_df, holdout_df])

# --- FIN DEL PROCESO ---

print("\n" + "-" * 45)
print("--- Conteo de clases por partición (DESPUÉS) ---")
print(final_df.groupby('partition')['class'].value_counts().sort_index())
print("-" * 45)

print("\nShape original del DataFrame:", df.shape)
print("Shape final del DataFrame:", final_df.shape)

--- Conteo de clases por partición (ANTES) ---
partition     class    
test          VS           454
              bogus        400
              asteroid     200
              AGN           52
              satellite     40
                          ... 
validation_4  bogus        320
              asteroid     160
              AGN           41
              satellite     32
              SN             7
Name: count, Length: 66, dtype: int64
---------------------------------------------
Procesando 'training_0': La clase minoritaria 'SN' tiene 29 objetos.
... Clase 'VS' (1452 obj): Reduciendo a un tamaño aleatorio en [29, 72] -> 39 objetos.
... Clase 'bogus' (1280 obj): Reduciendo a un tamaño aleatorio en [29, 72] -> 29 objetos.
... Clase 'asteroid' (640 obj): Reduciendo a un tamaño aleatorio en [29, 72] -> 72 objetos.
... Clase 'AGN' (164 obj): Reduciendo a un tamaño aleatorio en [29, 72] -> 41 objetos.
... Clase 'satellite' (128 obj): Reduciendo a un tamaño aleatorio en [29, 72] -

In [25]:
final_df[final_df.partition == 'training_0']['class'].value_counts()

class
asteroid     72
satellite    51
AGN          41
VS           39
bogus        29
SN           29
Name: count, dtype: int64